In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
# Load election data
data_dir = "../data/"
election_data = pd.read_csv(data_dir + "G20192-Columnes-ME-EN.csv", sep=';')

In [ ]:
# Load GeoJSON file
map_catalonia='municipality_map.geojson'
geo_data = gpd.read_file(data_dir + map_catalonia)

In [ ]:
geo_data.rename(columns={'nom_muni': 'Municipality_name'}, inplace=True)
geo_data_sorted=geo_data.sort_values(by='Municipality_name')

In [ ]:
election_data

In [ ]:
participation_by_municipality = election_data.iloc[:, 11:21].groupby(by=election_data["Municipality_name"]).sum()
participation_by_municipality.reset_index()
participation_by_municipality

In [ ]:
participation_by_municipality['Final part/Census'] = participation_by_municipality['Part_20:00'] / participation_by_municipality['Electoral_census']
participation_by_municipality

In [ ]:
merged_data = geo_data_sorted.merge(participation_by_municipality, on='Municipality_name')
merged_data

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(
    merged_data,
    geojson=merged_data.geometry,
    locations=merged_data.index,
    color='Final part/Census',  # Change to the column you want to visualize
    hover_name='Municipality_name',
    mapbox_style="carto-positron",
    center={"lat": 41.8781, "lon": 1.7834},  # Center of Catalonia
    zoom=7
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(height=600, width=800)
fig.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot a scatter plot
plt.scatter(participation_by_municipality['Electoral_census'], participation_by_municipality['Final part/Census'])
plt.xlabel('Electoral_census')
plt.ylabel('Voters/Intitled')
plt.xscale('log')  # Set x-axis to log scale
#plt.yscale('log')  # Set y-axis to log scale

# Show the plot
plt.show()

# Calculate and print the correlation coefficient
correlation_coefficient = participation_by_municipality['Electoral_census'].corr(participation_by_municipality['Final part/Census'])
print(f'Correlation Coefficient: {correlation_coefficient}')

In [ ]:
plt.hist(participation_by_municipality['Final part/Census'], bins=30)